In [3]:
!pip install pysolr pandas

     |████████████████████████████████| 9.3 MB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 109.5 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 110.6 MB/s eta 0:00:01


In [4]:
from sys import path
from query import Query
from client import solrClient
import pandas as pd
import re
import os

In [5]:
punctuation ='\d)@#$%&*.!,;:?"\''
def removePunctuation(text):
    text = re.sub(r'[{}]+'.format(punctuation), '', text)
    return text.strip().lower()

In [6]:
# path to quries file and dictionary for reserve results
queries_path = '~/playground/projects/fangyubo/pyserini/collections/info_needs.csv'
results_path = '~/playground/projects/fangyubo/pyserini/runs/results_v4'

In [7]:
# load in queries
queries = pd.read_csv(queries_path)
queries

,qid,query_text,len_label,readability
0,1048585,what is paula deen's brother,short,fairly diffcult
1,2,Androgen receptor define,long,extremely diffcult
2,524332,treating tension headaches without medication,short,fairly diffcult
3,1048642,what is paranoid sc,equal,diffcult
4,524447,treatment of varicose veins in legs,long,diffcult
...,...,...,...,...
7432,734979,what is coronary cta,long,diffcult
7433,524166,transportation to us bank stadium,equal,easy
7434,968921,where did last names originate,equal,plain
7435,786375,what is preoperative clearance,equal,diffcult


In [11]:
# perform searching using queries and reserve results.
results = pd.DataFrame(columns=['qid', 'pid', 'passage', 'len_label', 'readability', 'rank'])
for i in range(queries.shape[0]):
    client = solrClient()
    query = queries.loc[i, :]
    qid = query['qid']
    query_fields = ['contents', 'len_label', 'readability']
    query_values = [removePunctuation(query['query_text']), query['len_label'], query['readability']]
    q = Query('field_query', qid, query_fields, query_values)
    try:
        client.get_results(q)
    except:
        continue
    client.export_results(path=f"{results_path}/run.csv")
    results = results.append(pd.DataFrame(client.results))
    print(f"{i} queries processed.")
    

search query 1048585
found 0 result(s)
-------------------------------------------------
0 queries processed.
search query 2
found 0 result(s)
-------------------------------------------------
1 queries processed.
search query 524332
found 0 result(s)
-------------------------------------------------
2 queries processed.
search query 1048642
found 30624 result(s)
-------------------------------------------------
3 queries processed.
search query 524447
found 16384 result(s)
-------------------------------------------------
4 queries processed.
search query 524447
found 4310 result(s)
-------------------------------------------------
5 queries processed.
search query 524447
found 6909 result(s)
-------------------------------------------------
6 queries processed.
search query 786674
found 0 result(s)
-------------------------------------------------
7 queries processed.
search query 1048876
found 0 result(s)
-------------------------------------------------
8 queries processed.
search 

In [16]:
results

,qid,pid,passage,len_label,readability,rank
0,1048585,Null,Null,short,fairly diffcult,Null
0,2,Null,Null,long,extremely diffcult,Null
0,524332,Null,Null,short,fairly diffcult,Null
0,1048642,3163863,Related Search Terms: What is a centrifugal pu...,equal,diffcult,1
1,1048642,8703288,What is the opposite of rectifies? What is the...,equal,diffcult,2
...,...,...,...,...,...,...
995,1048565,3868775,We do not yet know what makes the enzymes atta...,equal,plain,996
996,1048565,3881995,"The cuisine had its origins with the Taino, wh...",equal,plain,997
997,1048565,3884765,Then-President Richard Nixon called Truman a m...,equal,plain,998
998,1048565,3900666,Confidence votes 4.2K. Answer: All American In...,equal,plain,999


In [17]:
run = results[['qid', 'pid', 'rank']]
run

,qid,pid,rank
0,1048585,Null,Null
0,2,Null,Null
0,524332,Null,Null
0,1048642,3163863,1
1,1048642,8703288,2
...,...,...,...
995,1048565,3868775,996
996,1048565,3881995,997
997,1048565,3884765,998
998,1048565,3900666,999


In [18]:
# drop all the results which no documents have been retrieved.
run = run.drop(run[run['pid'] == 'Null'].index)
run

,qid,pid,rank
1,1048642,8703288,2
2,1048642,4421230,3
3,1048642,513754,4
4,1048642,7679732,5
5,1048642,3806236,6
...,...,...,...
995,1048565,3868775,996
996,1048565,3881995,997
997,1048565,3884765,998
998,1048565,3900666,999


In [20]:
run.to_csv(f'{results_path}/exp2_run.tsv', sep='\t', index=0, header=0)

In [1]:
#experiment 2 results measure
!python3 ~/work/anserini/tools/scripts/msmarco/msmarco_eval.py \
~/shared/Datasets/MSMARCO/MSMARCO-Deep-Learning/qrels.dev.small.tsv \
~/playground/projects/fangyubo/pyserini/runs/results_v4/exp2_run.tsv

Cannot rank a passage multiple times for a single query. QID=98847, PID=5305092
#####################
MRR @10: 0.0040950902806203655
QueriesRanked: 3592
#####################


In [2]:
# original BM25 results measure
!python3 ~/work/anserini/tools/scripts/msmarco/msmarco_eval.py \
~/shared/Datasets/MSMARCO/MSMARCO-Deep-Learning/qrels.dev.small.tsv \
~/playground/projects/fangyubo/pyserini/runs/run.msmarco-passage.dev.small.tsv

#####################
MRR @10: 0.18741227770955546
QueriesRanked: 6980
#####################


In [4]:
#experiment 1 results measure
!python3 ~/work/anserini/tools/scripts/msmarco/msmarco_eval.py \
~/shared/Datasets/MSMARCO/MSMARCO-Deep-Learning/qrels.dev.small.tsv \
~/playground/projects/fangyubo/pyserini/runs/results_v2_clear/results_all_clear_without_null.tsv

#####################
MRR @10: 0.0025252421885659697
QueriesRanked: 6917
#####################
